<a href="https://colab.research.google.com/github/YuansongFeng/MadMario/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre-MVP tutorial for walking users through building a learning Mario. Guidelines for creating this notebook (feel free to add/edit):
1. Extensive explanation (link to AI cheatsheet where necessary) 
2. Only ask for core logics


In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 
!pip install gym-super-mario-bros==7.3.0 > /dev/null 

ERROR: nes-py 8.1.4 has requirement pyglet>=1.5.5, but you'll have pyglet 1.5.0 which is incompatible.
ERROR: gym 0.17.2 has requirement pyglet<=1.5.0,>=1.4.0, but you'll have pyglet 1.5.7 which is incompatible.


In [ ]:
from IPython.display import HTML
from IPython import display as ipythondisplay
import glob
import io
import base64

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

# Welcome to Mad Mario! 

We put together this project to walk you through fundamentals of reinforcement learning. Along the project, you will implement a smart Mario that learns to complete levels on itself. To begin with, you don't need to know anything about reinforcement learning. In case you wanna peek ahead, here is a [cheatsheet on RL basics](https://drive.google.com/file/d/1qwyemCX7o37OLbSky7Kkufg_YViWwL-g/view?usp=sharing) that we will refer to throughout the project. At the end of the tutorial, you will gain a solid understanding of RL fundamentals and implement a classic RL algorithm, Q-learning, on yourself. 


It's recommended that you have familiarity with Python and high school level of math/stats background -- that said, don't worry if memory is blurry. Just leave comments anywhere you feel confused, and we will explain the section in more details. 

## Let's get started! 

First thing first, let's look at what we will build: Just like when we first try the game, Mario enters the game not knowing anything about the game. It makes random action just to understand the game better. Each failure experience adds to Mario's memory, and as failure accumulates, Mario starts to recognize the better action to take in a particular scenario. Eventually Mario learns a good strategy and completes the level. 

Let's put the story into pseudo code.

```
for a total of N episodes:
  for a total of M steps in each episode:
    Mario makes an action
    Game gives a feedback 
    Mario remembers the action and feedback
    after building up some experiences:
      Mario learns from experiences   
```



In RL terminology: agent (Mario) interacts with environment (Game) by choosing actions, and environment responds with reward and next state. Based on the collected (state, action, reward) information, agent learns to maximize its future return by optimizing its action policy. 

While these terms may sound scary, in a short while they will all make sense. It'd be helpful to review the [cheatsheet](https://drive.google.com/file/d/1qwyemCX7o37OLbSky7Kkufg_YViWwL-g/view?usp=sharing), before we start coding. 



# Environment
Environment is a key concept in reinforcement learning. It's the world that Mario interacts with and learns from. Environment is characterized by state (cheatsheet). In Mario, this is the game console consisting of tubes, mushrooms and other components. When Mario makes an action, environment responds with a reward (cheatsheet) and the next state (cheatsheet).  


To run Mario envirioment, do






In [ ]:
import gym_super_mario_bros
from gym.wrappers import Monitor
from nes_py.wrappers import JoypadSpace


env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(
    env,
    [['right'],
     ['left'],
    ['right', 'A'],
    ['left', 'A']]
    )
env = Monitor(env, './video', force=True, mode = 'evaluation')
env.reset()
for _ in range(1000):
  # render game output
  env.render()

  # choose random action
  action = env.action_space.sample()

  # perform action on environment, environment provides feedback 
  # with env.step() function
  env.step(action=action)

env.close()

show_video()

## Wrappers

Like in supervised machine learning, data preparation is crucial in reinforcement learning. For example, a standard pre-processing step is turning RGB image/videos to grayscale. This compresses input size without losing much information. 

In the previous section we talked about environment in general. However, in many cases we want to do some pre-processing to the environment and its data before we feed them to the agent. This introduces the idea of a **wrapper**.

Like its name suggests, a wrapper wraps around the environment and modifies its methods/data in some generic way. For example, the Mario environment gives you some observations, but you want to accumulate them in some buffer and provide to the agent the N last observations, which is a common scenario for dynamic computer games like Mario, when one single frame is just not enough to get full information about the game state. For a concrete example, we cannot determine if Mario is in a jumping or landing motion from just one frame of environment output. We need some previous frames to have that insight.

Another common wrapper is to transform RGB picture frames into Grayscale. It makes sense because as far as the agent is concerned, the outcome of the game or how much reward it collects doesn't change whether he lives in a RGB world or Grayscale world!

**before pre-processing**

![picture](https://drive.google.com/uc?id=1c9-tUWFyk4u_vNNrkZo1Rg0e2FUcbF3N)
<!-- ![picture](https://drive.google.com/uc?id=1s7UewXkmF4g_gZfD7vloH7n1Cr-D3YYX)
![picture](https://drive.google.com/uc?id=1mXDt8rFLKT9a-YvhGOgGZT4bq0T2y7iw) -->

**after pre-processing**

![picture](https://drive.google.com/uc?id=1ED9brgnbPmUZL43Bl_x2FDmXd-hsHBQt)
<!-- ![picture](https://drive.google.com/uc?id=1PB1hHSPk6jIhSxVok2u2ntHjvE3zrk7W)
![picture](https://drive.google.com/uc?id=1CYm5q71f_OlY_mqvZADuMMjPmcMgbjVW) -->

We perform pre-processing on state through environment wrappers. By wrapping `env` like so 
```
env = wrapper(env)
```
states from environment are transformed into our desired format.  


### Instructions

We want to apply 3 built-in wrappers to the given `env`, `GrayScaleObservation`, `ResizeObservation`, and `FrameStack`.  

https://github.com/openai/gym/tree/master/gym/wrappers


`FrameStack` is a wrapper that will allow us to squash consecutive frames of the environment into a single observation point to feed to our learning model. This way, we can differentiate between when Mario was landing or jumping based on his direction of movement in the previous several frames. 

We can start with the following arguments:
`GrayScaleObservation`: keep_dim=False 
`ResizeObservation`: shape=84 
`FrameStack`: num_stack=4 




In [ ]:
from gym.wrappers import FrameStack, GrayScaleObservation, ResizeObservation
import gym_super_mario_bros

# the original environment object 
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')

# TODO wrap the given env with GrayScaleObservation
env = None
# TODO wrap the given env with ResizeObservation
env = None
# TODO wrap the given env with FrameStack 
env = None

## Custom Wrapper

We also would like you to get a taste of implementing an environment wrapper on your own, instead of calling off-the-shelf packages. 

Here is an idea:
As an effort of downsizing our model to make training faster, we can choose to skip every n-th frame. In other words, our wrapped environment will only output every n-th frame. Below is a skeleton of the class `SkipFrame`, inherited from `gym.Wrapper`.  Notice in the `__init__` function, the `_skip` field is overriden by the input parameter, default set at 4.

However, it is important to accumulate the reward during these skipped steps, because the reward is the most important factor in determining the success of the learning model, so while we can skip frame for dimension reduction purpose, it is crucial we keep adding those rewards to our total reward. 


### Instruction

Implement the reward accumulation function, using your favorite `for` loop.  

Here, the custom `SkipFrame` inherits `gym.Wrapper`, the superclass of all wrappers we are going to use. We are going to override its `step()` method, which is basically the endpoint that environment interacts with agent:

Given an `action`, env.step() returns a tuple of `next_state(obs), reward, done, info`

In [ ]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # TODO accumulate reward and repeat the same action 
            obs, reward, done, info = None
            if done:
                break

        return obs, total_reward, done, info
       
env = SkipFrame(env)

# Agent

Agent is the other core concept in reinforcement learning. It interacts with environemnt by making actions (link to cheatsheet). The agent acts according to its action policy (link to cheatsheet). 






```
Initialize environment
While True:
  ### agent chooses an action 
  ### based on current state
  agent.act(state)
  ### environment provides feedback/reward 
  ### and state transition
  next_state, reward, done, info = env.step(action=action)
  state = next_state
  if done: 
    break

```

The agent class, `Mario`, captures Mario's behavior in the game environment. The agent should be able to 

- Make its decision about next action to take. Mario takes the environment state and acts following its optimal action (link to cheatsheet) 

- Remember past experiences. Later mario uses these experiences to update its action policy (link to cheatsheet)

- Improve action policy over time. Mario updates its action policy following the Q-learning algorithm (link to cheatsheet). 

In [ ]:
class Mario:
    def __init__(self, state_dim, action_dim, max_memory):
        pass

    def act(self, state):
        """Given a state, choose an epsilon-greedy action
        """
        pass

    def remember(self, experience):
        """Add the observation to memory
        """
        pass

    def learn(self):
        """Update online action value (Q) function with a batch of experiences
        """
        pass


## Initialize
Before implementing the core functions, let's define some key parameters. For example, we use *epsilon* in action policy (refer to cheatsheet) to encourage exploration, use *gamma* to calculate discounted future return (link to cheatsheet). 

eps, float
> Random Exploration Prabability. Under some probability, agent does not follow the optimal action policy (cheatsheet), but instead chooses a random actino to explore the environment. A high exploration rate is important at the early stage of learning to ensure proper exploration and not falling to local optima. The exploration rate should decrease as agent improves its policy. 

> Initialize to 1.0. 



eps_decay, float

> Decay rate of eps. Agent rigorously explores space at the early stage, but gradually reduces its exploration rate to maintain action quality. In the later stage, agent already learns a fairly good policy, so we want it to follow its policy more frequently. Decrease eps by the factor of eps_decay each time the agent acts.

> Initialize to 0.99999975. 


gamma, float
> Future reward discount rate. *gamma* serves to make agent give higher weight on the short-term rewards over future reward.

> Initialize to 0.9

batch_size, int
> Number of experiences used to update each time.

> Please initialize it to 32

state_dim, tuple of int

> State space dimension. In Mario, this is 4 consecutive snapshots of the enviroment stacked together, where each snapshot is a 84*84 gray-scale image, so state_dim = (4, 84, 84).

action_dim, int
>  Action space dimension. In Mario, this is the number of total possible actions. 

max_memory, int
> Size of Mario's memory. The memory is filled with Mario's past experiences. Each experience consists of (state, next_state, action, reward, done). Note we use a *queue* (FIFO) for storing memory. As Mario collects more experiences, old experiences are popped to make room for most recent ones. 

In [ ]:
class Mario:
    def __init__(self, state_dim, action_dim, max_memory):
       # state space dimension
      self.state_dim = state_dim
      # action space dimension
      self.action_dim = action_dim
      # replay buffer
      self.memory = deque(maxlen=max_memory)
      # current step, updated everytime the agent acts
      self.step = 0

      # TODO: Please initialize other variables as described above
      pass
    


## Predict Q value

*Q(s, a)*, optimal value function of a state-action pair, is the most important function in this project. It's used to both choose optimal action (cheatsheet) and to improve action policy (cheatsheet to q-learning). Let's implement the method `agent.predict()`. 


### Define Q function

As a reminder, `env.step(action)` is the endpoint for which the environment provides feedback to the agent's action. Its output is of type `LazyFrame`, and we need to normalize it into an numpy array before doing any more processing. 

Because input to *Q(s, a)* is a stack of images(`LazyFrame`), let's use a *convolution neural network(ConvNet)* as our Q function. Instead of passing action together with state into the Q function, we pass only state to the ConvNet, and it would returns a list of real values representing Q values og=f all possible actions. We then choose the specific Q value based on the given action. 

Let's now look at Q-learning more closely. We use *Q\*(s, a)* to define both the *TD target* (r + \gamma max_a Q(s', a')) and current state-action value (Q(s, a)). While mathematically Q(s', a') and Q(s, a) are all the same Q function, in practice we use a separate function to approximate each. This is to prevent the divergence problem during optimization. We use Q_online to represent the Q(s, a) function, and Q_target to represent the Q(s', a') function. Q_online is used to make actual action decision by agent in `agent.act()`. Q_target is used to determine the optimization target for Q_online. 

### Instructions

We have implemented `ConvNet`, a simple convolution neural network, for you. Define `self.online_q` and `self.target_q` as two separate `ConvNet`s. `ConvNet` takes two parameters: input dimension `input_dim` and output dimension `output_dim`. Initialize input dimension with state dimension and output dimension with action dimension. 

Example:
```
self.neural_net = ConvNet(input_dim=state_dim, output_dim=action_dim)
```

In [ ]:
class Mario:
    def __init__(self, ...):
      # TODO: define online action value function
      self.online_q = None
      # TODO: define target action value function
      self.target_q = None

### Call Q function

### Instruction 

Q function takes a single input `state`. Depending on the requested model ('online'/'target'), use the corresponding Q functons to calculate Q values of the given `state`. Note the result is Q values for all possible actions. 

Example:
```
q_values = q_function(state)
```

In [ ]:
class Mario:
      def predict(self, state, model):
        """Given a state, predict Q values of all possible actions using specified model (either online or target)
        Input:
          state
           dimension of (batch_size * state_dim)
          model
           either 'online' or 'target'
        Output
          pred_q_values (torch.tensor)
            dimension of (batch_size * action_dim), predicted Q values for all possible actions given the state
        """
        # LazyFrame -> np array -> torch tensor
        state_float = torch.FloatTensor(np.array(state))
        # normalize
        state_float = state_float / 255.
        
        if model == 'online':
          # TODO return the predicted Q values using self.online_q
          pred_q_values = None
        elif model == 'target':
          # TODO return the predicted Q values using self.target_q
          pred_q_values = None

        return pred_q_values

## Act

Let's now look at how Mario should `act()` in the environment. 

Given a state, Mario mostly chooses the action with the highest Q value (cheatsheet to optimal action). There is an *epislon* chance that Mario acts randomly instead, which encourages environment exploration. 

### Instruction

We will now implement `Mario.act()`. Recall that we have defined Q_online above, which we will use here to calculate Q values for all actions given *state*. We then need to select the action that results in largest Q value. We have set up the logic for epsilon-greedy policy, and leave it to you to determine the optimal and random action. 

Before implementing `Mario.act()`, let's first get used to basic operations on *torch.tensor*, which is the data type returned in `Mario.predict()`
     

###    Get familiar with torch.tensor:

1. To create a tensor

In [ ]:
import torch
import torch.tensor as tensor
t = tensor([[1,2,3],[8,9,4],[7,6,5]])
t

tensor([[1, 2, 3],
        [8, 9, 4],
        [7, 6, 5]])

2. To get element at 1st row, 2nd col:

In [ ]:
t[0,1]


tensor(2)

3. To get elements at (1st row, 2nd col) and (3rd row, 1st col):

In [ ]:
t[[0,2],[1,0]]

tensor([2, 7])

4. To get the largest number:

In [ ]:
torch.max(t)

tensor(9)

5. To get largest values and corresponding indices along 1st axis:

In [ ]:
torch.max(t, axis = 0)

torch.return_types.max(values=tensor([8, 9, 5]), indices=tensor([1, 1, 2]))


> Note: From the example above, you can see torch.max return a *namedtuple (values, indices)*, where *values* contains the maximum number along the given axis. And *indices* contain the indices location of each maximum value found (argmax). Thus, you do not need a different function to get the argmax.




6. To get largest values along 1st axis:

In [ ]:
torch.max(t, axis = 0)[0]

tensor([8, 9, 5])

7. To get indices of largest values along 1st axis:

In [ ]:
torch.max(t, axis = 0)[1]

tensor([1, 1, 2])

8. To convert torch.tensor to NumPy array:

In [ ]:
t.numpy()

array([[1, 2, 3],
       [8, 9, 4],
       [7, 6, 5]])

###    Get familiar with NumPy

1. Create a numpy array:

In [ ]:
import numpy as np
arr = np.array([[1,2,3],[8,9,4],[7,6,5]])
arr

array([[1, 2, 3],
       [8, 9, 4],
       [7, 6, 5]])

2. To get element at 1st row, 2nd col:

In [ ]:
arr[0,1]

2

3. To get elements at (1st row, 2nd col) and (3rd row, 1st col):

In [ ]:
arr[[0,2],[1,0]]

array([2, 7])

4. To get max numbers in a along 1st axis:

In [ ]:
np.max(arr, axis = 0)

array([8, 9, 5])

5. To get the indices of max numbers in a along 1st axis:

In [ ]:
np.argmax(arr, axis=0)

array([1, 1, 2])

Although some people are more comfortable using numpy array, sometimes we should not convert torch.tensor to numpy array, because torch.tensor has an attribute *grad* which keeps track of gradients for the tensor and is used to update our Q function(We will mention this later); converting it to numpy would lose gradient information

In [ ]:
class Mario:
    def act(self, state):
        """Given a state, choose an epsilon-greedy action and update value of step
        Input
          state(np.array) 
            A single observation of the current state, dimension is (state_dim)
        Output
          action
            An integer representing which action agent will perform
        """
        if np.random.rand() < self.eps:
          # TODO: choose a random action from all possible actions (self.action_dim)
          pass
        else:
          # TODO: choose the best action based on self.online_q
          pass
          
        # decrease eps
        self.eps *= self.eps_decay
        self.eps = max(self.eps_min, self.eps)
        # increment step
        self.step += 1
        return action

## Remember

In order to improve policy, Mario need to collect and save past experiences. Each time agent performs an action, it collects an experience which includes the current state, action it performs, the next state after performing the action, the reward it collected, and whether the game is finished or not. We use a Queue structure to save historic experience, consisting of (state, next_state, action, reward, done). We will refer to this Queue as our memory. 

### Instruction

Implement `Mario.remember()` to save the experience to Mario's memory. 

In [ ]:
class DQNAgent:
    def remember(self, experience):
        """Add the experience to self.memory
        Input
          experience =  (state, next_state, action, reward, done) tuple
        Output
            None
        """
        # TODO Add the experience to memory
        pass

## Learn


The entire learning process is based on Q-learning algorithm (cheatsheet). By learning, we mean updating our Q function to better predict the Q value of given state-action pair. Recall that Q_online is the function that we are updating and using to choose actions. Q_target is used to determine the target Q value.  


Some key steps to perform:
- Experience Sampling: 
We will sample experiences from memory as the “training set” of the current learning step. 


- Predicting Online Q Values: 
We predict Q values for all sampled state-action pairs using Q_online.


- Predicting Target Q Values:
We predict Q values for all sampled state-action pairs using Q_target and reward. Unlike predicting online Q values, we don't directly calculate the Q value for the state-action pair. Instead we approximate it with the sum of reward and discounted Q value for the next state-action pair (cheatsheet). 


- Loss between Online and Target Q:
In this step, we calculate the loss between predicted online Q values and target Q values. 


- Update Q_online: 
Update Q_online to minimize the loss using Adam optimizer. This improves the predicting accuracy of Q_online. 


Summarizing the above in pseudo code for `Mario.learn()`:

```
if enough experiences are collected:
  sample a batch of experiences
  calculate the predicted Q values using Q_online
  calculate the target Q values using Q_target and reward
  calculate loss between prediction and target Q values
  update Q_online based on loss
```

### Experience Sampling 
Mario learns by drawing past experiences from its memory. The memory is a queue data structure that stores each individual experience in the format of 

> state, next_state, action, reward, done

Examples of some experiences in Mario's memory:


- state: ![pic](https://drive.google.com/uc?id=1D34QpsmJSwHrdzszROt405ZwNY9LkTej)  next_state: ![pic](https://drive.google.com/uc?id=13j2TzRd1SGmFru9KJImZsY9DMCdqcr_J) action: jump reward: 0.0 done: False


- state: ![pic](https://drive.google.com/uc?id=1ByUKXf967Z6C9FBVtsn_QRnJTr9w-18v) next_state: ![pic](https://drive.google.com/uc?id=1hmGGVO1cS7N7kdcM99-K3Y2sxrAFd0Oh) action: right reward: -10.0 done: True


- state: ![pic](https://drive.google.com/uc?id=10MHERSI6lap79VcZfHtIzCS9qT45ksk-) next_state: ![pic](https://drive.google.com/uc?id=1VFNOwQHGAf9pH_56_w0uRO4WUJTIXG90) action: right reward: -10.0 done: True


- state: ![pic](https://drive.google.com/uc?id=1T6CAIMzNxeZlBTUdz3sB8t_GhDFbNdUO) next_state: ![pic](https://drive.google.com/uc?id=1aZlA0EnspQdcSQcVxuVmaqPW_7jT3lfW) action: jump_right reward: 0.0 done: False


- state: ![pic](https://drive.google.com/uc?id=1bPRnGRx2c1HJ_0y_EEOFL5GOG8sUBdIo) next_state: ![pic](https://drive.google.com/uc?id=1qtR4qCURBq57UCrmObM6A5-CH26NYaHv) action: right reward: 10.0 done: False

State/next_state:
Observation at timestep *t*/*t+1*. They are both of type `LazyFrame`. 

Action:
Mario's action during state transition. 

Reward:
Environment's reward during state transition. 

Done:
Boolean indicating if next_state is a terminal state (end of game). Terminal state has a known Q value of 0. 


## Instruction

Sample a batch of experiences from `self.memory` of size `self.batch_size`. 

Return a tuple of numpy arrays, in the order of (state, next_state, action, reward, done). Each numpy array should have its first dimension equal to `self.batch_size`. 

To convert a `LazyFrame` to numpy array, do

```
state_np_array = np.array(state_lazy_frame)
```

 

In [ ]:
class Mario:
  def sample_batch(self, batch_size):
    """
    Input
      self.memory (FIFO queue)
        a queue where each entry has five elements as below
        state: LazyFrame of dimension (state_dim)
        next_state: LazyFrame of dimension (state_dim)
        action: integer, representing the action taken
        reward: float, the reward from state to next_state with action
        done: boolean, whether state is a terminal state
      batch_size (int)
        size of the batch to return 

    Output
      state, next_state, action, reward, done (tuple)
        a tuple of numpy arrays: state, next_state, action, reward, done
        state: numpy array of dimension (batch_size x state_dim)
        next_state: numpy array of dimension (batch_size x state_dim)
        action: numpy array of dimension (batch_size)
        reward: numpy array of dimension (batch_size)
        done: numpy array of dimension (batch_size)
    """
    return None, None, None, None, None

### Prediction Q Value

The learning process relies on the Q-learning algorithm (cheatsheet). We refer to the calculated Q values using current state-action pair as *prediction Q values*. These are calculated using Q_online. 

To call Q_online, use our defined `Mario.predict()` above:
```
q_values = self.predict(state, model='online')
```


## Instruction

For a batch of experiences consisting of states (s) and actions (a), calculate the estimated value for each state-action pair Q(s, a). Return the results in `torch.tensor` format. 


In [ ]:
class Mario:  
  def calculate_prediction_q(state, action):
    """
    Input
      state (np.array)
        dimension is (batch_size x state_dim), each item is an observation 
        for the current state 
      action (np.array)
        dimension is (batch_size), each item is an integer representing the 
        action taken for current state 

    Output
      pred_q (torch.tensor)
        dimension of (batch_size), each item is a predicted Q value of the 
        current state-action pair 
    """
    curr_state_q = self.predict(state, model='online')
    # TODO select specific Q values based on input actions 
    curr_state_q = None

    return curr_state_q

### Target Q Value

We refer to the calculated Q values using current reward and next state-action pair as *target Q values* (cheatsheet). *target Q values* are in the form of 

> r + γ max Q(s', a')

A small caveat is the terminal state, as recorded with the variable `done`, which is 1 when Mario is dead or the game finishes. 

Hence, we need to make sure we don't keep adding future rewards when "there is no future", i.e. when the game reaches terminal state. Since `done` is a boolean, we can multiply `1. - done` with future reward. This way, future reward after the terminal state is not taken into account in our target Q value calculation.

The complete *target Q values* are in the form of 

> r + (1- done)⋅γ max Q(s', a')

Q is the Q_target function, r is the current reward, s' is the next state, a' is the next action. Because we don't know what next action will be, we estimate it using next state s' and Q_online. Specifically,

> a' = argmax_a Q_online(s', a)

Let's calculate *target Q values* now. 

## Instruction

For a batch of experiences consisting of next_states (s') and rewards (r), calculate the target Q values. Note that a' is not explicitly given, so we will need to first obtain that using Q_online and next state s'.

Return the results in `torch.tensor` format. 

In [ ]:
class Mario:  
  def calculate_target_q(next_state, reward):
    """
    Input
      next_state (np.array)
        dimension is (batch_size x state_dim), each item is an observation 
        for the next state 
      reward (np.array)
        dimension is (batch_size), each item is a float representing the 
        reward collected from (state -> next state) transition 

    Output
      target_q (torch.tensor)
        dimension of (batch_size), each item is a target Q value of the current
        state-action pair, calculated based on reward collected and 
        estimated Q value for next state
    """
    next_state_q = self.predict(next_state, 'target')

    online_q = self.predict(next_state, 'online')
    # TODO select the best action at next state based on online Q function
    action_idx = None

    # TODO calculate target Q values based on action_idx and reward
    target_q = None
    
    return target_q

### Loss

Let's now calculate the loss between *prediction Q values* and *target Q values*. Loss is what drives the optimization and updates Q_online to better predict Q values in the future. We will calculate the mean squared loss as:

$MSE = \frac{1}{n}\sum_{i=0}^n( y_i - \hat{y}_i)^2$

In Pytorch, we can calculate loss between *prediction Q values* and *target Q values* simply using:
```
loss = nn.functional.mse_loss(pred, target)
```

## Instruction

Given predicted and target Q values for the batch of experiences, return the Mean Squared Error. 


In [ ]:
class Mario:
  def calculate_loss(pred_q, target_q):
    """
    Input
      pred_q (torch.tensor)
        dimension is (batch_size), each item is an observation 
        for the next state 
      target_q (torch.tensor)
        dimension is (batch_size), each item is a float representing the 
        reward collected from (state -> next state) transition 

    Output
      loss (torch.tensor)
        a single value representing the MSE loss of pred_q and target_q
    """
    return None

### Update Q_online

As the final step to complete `Mario.learn()`, we use Adam optimizer to optimize upon the above calculated `loss`. This updates the parameters inside Q_online function so that prediction Q values are closer to target Q values.  

In [ ]:
class Mario:
  def __init__(self, ...):
    # optimizer updates parameters in online_q using backpropagation
    self.optimizer = torch.optim.Adam(self.online_q.parameters(), lr=0.00025)

  def update_online_q(self, loss):
    '''
    Input
      loss (torch.tensor)
        a single value representing the Huber loss of pred_q and target_q
      optimizer
        optimizer updates parameter in our online_q neural network to reduce
        the loss
    '''
    # update online_q
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    


### Put them Together
With all the helper methods implemented, let's revisit our `Mario.learn()` function. 

### Instructions

We've added some logic on checking learning criterion. For the rest, use the helper methods defined above to complete `Mario.learn()` function. 

In [ ]:
class Mario:
    def learn(self):
        """Update prediction action value (Q) function with a batch of experiences
        """
        # sync target network
        if self.step % self.copy_every == 0:
            self.sync_target_q()
        # checkpoint model
        if self.step % self.save_every == 0:
            self.save_model()
        # break if burn-in
        if self.step < self.burnin:
            return
        # break if no training
        if self.step % self.learn_every != 0:
            return

        # TODO: sample a batch of experiences from self.memory
        state, next_state, action, reward, done = (None, None, None, None, None)

        # TODO: calculate prediction Q values for the batch
        pred_q = None

        # TODO: calculate target Q values for the batch
        target_q = None

        # TODO: calculate huber loss of target and prediction values
        loss = None
        
        # TODO: update target network
        pass


## Completed Agent

We now have all the key functionlities implemented! Glory is all yours. Here is the completed agent file, revisited. 

In [ ]:
from collections import deque
import torch
import torch.nn as nn
import numpy as np
import random
from neural import ConvNet
import pdb

class DQNAgent:
    def __init__(self, state_dim, action_dim, max_memory, double_q):
        # state space dimension
        self.state_dim = state_dim
        # action space dimension
        self.action_dim = action_dim
        # replay buffer
        self.memory = deque(maxlen=max_memory)
        # if double_q, use best action from online_q for next state q value
        self.double_q = double_q
        # future reward discount rate
        self.gamma = 0.9
        # initial epsilon(random exploration rate)
        self.eps = 1
        # final epsilon
        self.eps_min = 0.1
        # epsilon decay rate
        self.eps_decay = 0.99999975
        # current step, updated everytime the agent acts
        self.step = 0
        # number of experiences between updating online q
        self.learn_every = 3
        # number of experiences to collect before training
        # self.burnin = 1e5
        self.burnin = 1e2
        # number of experiences between updating target q with online q
        self.copy_every = 1e4
        # number of experiences between saving the current agent
        self.save_every = 5e5

        # batch size used to update online q
        self.batch_size = 32
        # online action value function, Q(s, a)
        self.online_q = ConvNet(input_dim=state_dim, output_dim=action_dim)
        # target action value function, Q'(s, a)
        self.target_q = ConvNet(input_dim=state_dim, output_dim=action_dim)
        # optimizer
        self.optimizer = torch.optim.Adam(self.online_q.parameters(), lr=0.00025)

    def predict(self, state, model):
        """Given a state, predict Q values of all actions
        model is either 'online' or 'target'
        """
        state_float = torch.tensor(np.array(state)).float() / 255.
        if model == 'online':
            return self.online_q(state_float)
        if model == 'target':
            return self.target_q(state_float)

    def act(self, state):
        """Given a state, choose an epsilon-greedy action
        """
        if np.random.rand() < self.eps:
            # random action
            action = np.random.randint(low=0, high=self.action_dim)
        else:
            # policy action
            q = self.predict(np.expand_dims(state, 0), model='online')
            action = torch.max(q, axis=1)[1].item()
        # decrease eps
        self.eps *= self.eps_decay
        self.eps = max(self.eps_min, self.eps)
        # increment step
        self.step += 1
        return action

    def remember(self, experience):
        """Add the observation to memory
        """
        self.memory.append(experience)

    def learn(self):
        """Update online action value (Q) function with a batch of experiences
        """
        # sync target network
        if self.step % self.copy_every == 0:
            self.sync_target_q()
        # checkpoint model
        if self.step % self.save_every == 0:
            self.save_model()
        # break if burn-in
        if self.step < self.burnin:
            return
        # break if no training
        if self.step % self.learn_every != 0:
            return
        # sample batch
        batch = random.sample(self.memory, self.batch_size)
        state, next_state, action, reward, done = map(np.array, zip(*batch))
        # get next q values from target_q
        next_q = self.predict(next_state, 'target')
        # calculate discounted future reward
        if self.double_q:
            q = self.predict(next_state, 'online')
            q_idx = torch.max(q, axis=1)[1]
            target_q = torch.tensor(reward) + torch.tensor(1. - done) * self.gamma * next_q[np.arange(0, self.batch_size), q_idx]
        else:
            target_q = torch.tensor(reward) + torch.tensor(1. - done) * self.gamma * torch.max(next_q, axis=1)[0]
        # get predicted q values from online_q and actions taken
        curr_q = self.predict(state, 'online')
        pred_q = curr_q[np.arange(0, self.batch_size), action]
        # huber loss
        loss = nn.functional.mse_loss(pred_q, target_q)
        # update online_q
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def save_model(self):
        """Save the current agent
        """
        return

    def sync_target_q(self):
        """Update target action value (Q) function with online action value (Q) function
        """
        self.target_q.load_state_dict(self.online_q.state_dict())

# Start Learning! 

With the agent and environment wrappers implemented, we are ready to put Mario in the game and start learning! We will wrap the learning process in a big `for` loop that repeats the process of acting, remembering and learning by Mario. 

The meat of the algorithm is in the loop, let's take a closer look: 

### Instruction

1.At the beginning of a new episode, we need to reinitialize the `state` by calling `env.reset()`

2.Then we need several variables to hold the logging information we collected in this episode:

`ep_reward`: reward collected in this episode

`ep_num_steps`: total number of actions perforemed in this episode
    
`ep_total_loss`: total loss collected in this episode

`ep_total_q`: total loss collected in this episode

`ep_learn_length`: used for mean loss/q_value


3.Now we are inside the while loop that plays the game, and we can call `env.render()` to display the visual

4.We want to apply the action policy on the current state by calling `agent.act`, remember action policy  $pi$ characterizes how the agent reacts to environment.

5.Then after the agent performs the action, the environment will ouput its feedback, including information such as next state, reward, if Mario is dead(`done`), and some other info. Calling `env.step` with the agent's action should do the trick.

6.Agent needs to remember the experience in this action he takes in this state, call `agent.remember` using all the environment output from the step above.

7.Call `agent.learn`

8.Update logging info

9.Update state to get the evironment ready for the next interation of agent-environment/action-reward interactions


In [ ]:
### for Loop that train the model num_episodes times by playing the game

for e in range(num_episodes):

    # 1. Reset env/restart the game
    state = None

    # 2. Logging
    ep_reward = 0.0
    ep_length = 0
    ep_total_loss = 0.0
    ep_total_q = 0.0
    ep_learn_length = 1 # used for mean loss/q_value

    # Play the game!
    while True:

        # 3. Show environment (the visual)

        # 4. Run agent on the state
        action = None

        # 5. Agent performs action
        next_state, reward, done, info = None

        # 6. Remember
        agent.remember(experience=(None,None,None,None))

        # 7.Learn (conditional) (80% time cost)
        q_value, loss = None

        # 8.Logging
        ep_reward += reward
        ep_length += 1
        if q_value and loss:
            ep_total_loss += loss
            ep_total_q += q_value
            ep_learn_length += 1

        # Update state
        state = None

        # If done break loop
        if done or info['flag_get']:
            break



Below is the fully functional `main` class, we added logging info that will help keep track of the status of the training. 

We have helped you initialize and applied the wrappers for the environment for you, and we also initialized the agent.

In addition, we also added model saving functionality for you so that you can replay the model you trained.

In [ ]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
import numpy as np
import pdb
import time
# original environment
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')

# define action space on the environment:
# NOOP: no action
# right: walk right
# right, A: jump right
# right, B: run right
env = JoypadSpace(
    env,
    [['right'],
    ['right', 'A']]
    )

## apply environment wrappers
env = apply_wrappers_to_env(env)

# After applying environment wrappers, observation space (a.k.a state_dim) shrinks from
# 240 (height) x 256 (width) x 3 (RGB color channels) 
# to 
# 4 (#frames) x 84 (height) x 84 (width)

# dimensional parameters after reshaping
state_dim = (4,84,84)
action_dim = env.action_space.n

# Intialize agent
agent = DQNAgent(state_dim=state_dim, action_dim=action_dim, max_memory=100000, double_q=True)

# Logs
log = {
    "rewards": [],
    "lengths": [],
    "losses": [],
    "q_values": []
}
log_file = os.path.join(agent.save_dir, "log.txt")

# Timing
start = time.time()
step = 0

# number of episodes
num_episodes = 10000

# Main training loop
for e in range(num_episodes):

    # 1. Reset env/restart the game
    state = None

    # 2. Logging
    ep_reward = 0.0
    ep_length = 0
    ep_total_loss = 0.0
    ep_total_q = 0.0
    ep_learn_length = 1 # used for mean loss/q_value

    # Play the game!
    while True:

        # 3. Show environment (the visual)

        # 4. Run agent on the state
        action = None

        # 5. Agent performs action
        next_state, reward, done, info = None

        # 6. Remember
        agent.remember(experience=(None,None,None,None))

        # 7.Learn (conditional) (80% time cost)
        q_value, loss = None

        # 8.Logging
        ep_reward += reward
        ep_length += 1
        if q_value and loss:
            ep_total_loss += loss
            ep_total_q += q_value
            ep_learn_length += 1

        # Update state
        state = None

        # If done break loop
        if done or info['flag_get']:
            break

    # Log info in this episode
    log["rewards"].append(ep_reward)
    log["lengths"].append(ep_length)
    log["losses"].append(np.round(ep_total_loss/ep_learn_length, 5))
    log["q_values"].append(np.round(ep_total_q/ep_learn_length, 5))

    # Print & Log every 50th episode
    if e % 50 == 0:
        mean_reward = np.round(np.mean(log['rewards'][-100:]), 3)
        mean_length = np.round(np.mean(log['lengths'][-100:]), 3)
        mean_loss = np.round(np.mean(log['losses'][-100:]), 3)
        mean_q_value = np.round(np.mean(log['q_values'][-100:]), 3)
        eps = np.round(agent.eps, 3)
        step_time = np.round((time.time() - start_time)/(agent.step - start_step), 3)
        start_time = time.time()
        start_step = agent.step
        print(
            f"Episode {e} - "
            f"Step {agent.step} - "
            f"Step Time {step_time} - "
            f"Epsilon {eps} - "
            f"Mean Reward {mean_reward} - "
            f"Mean Length {mean_length} - "
            f"Mean Loss {mean_loss} - "
            f"Mean Q Value {mean_q_value} - "
            f"Time {datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')}"
        )

        with open(log_file, "a") as f:
            f.write(
                f"{e:8d}{agent.step:10d}{eps:10.3f}"
                f"{mean_reward:15.3f}{mean_length:15.3f}{mean_loss:15.3f}{mean_q_value:15.3f}"
                f"{datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'):>20}\n"
            )

        # Running on Colab, download checkpoints to local
        if 'google.colab' in sys.modules:
            from google.colab import files
            files.download(os.path.join(agent.save_dir, "online_q_1.chkpt"))

# Discussion

One question one might ask why do we want to sample data points from all past experiences rather than the most recent ones(for example, from the previous episode), which are newly trained with higher accuracy. 

The intuition is behind the tradeoff between these two approaches:

Do we want to train on data that are generated from a small-size dataset with relatively high quality or a huge-size dataset with relatively lower quality? 

The answer is the latter, because the more data we have, the more of a wholistic, comprehensive point of view we have on the overall behavior of the system we have, in our case, the Mario game. Limited size dataset has the danger of overfitting and overlooking bigger pictures of the entire action/state space. 


Remember, Reinforcement Learning is all about exploring different scenarios(state) and keeping improving based on trial and errors, generated from the interactions between the **agent**(action) and the **environmental feedback**(reward). 